<a href="https://colab.research.google.com/github/Aftaza/Sistem-Automasi-Tkom/blob/master/Fungsi_Alih_Motor_Dc_Tugas_Sistem_Automasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tugas Pembuatan Kode Fungsi Alih Motor DC
## Sistem Automasi
> M Alfian Taftazani  
> 225150300111007

### Inisiasi

In [1]:
import sympy as sp
from sympy import symbols, Eq, solve, laplace_transform, inverse_laplace_transform

# Definisikan simbol-simbol
t, s = symbols('t s')  # waktu dan variabel Laplace
V, I, omega = symbols('V I omega', real=True)  # Tegangan, Arus, Kecepatan sudut
R, L, J, B, K_t, K_e = symbols('R L J B K_t K_e', positive=True)  # Parameter motor
theta = sp.Function('theta')(t)  # Posisi sudut sebagai fungsi waktu
i = sp.Function('i')(t)  # Arus sebagai fungsi waktu
v = sp.Function('v')(t)  # Tegangan sebagai fungsi waktu

### 1. Model Matematika Motor DC (Persamaan Listrik dan Mekanik)
- Listrik: $V = R ⋅ I + L ⋅ \frac{di}{dt} + K_e ⋅ ω$
- Mekanik: $T = J ⋅ \frac{dω}{dt} + B ⋅ ω$, dengan $T = K_t ⋅ I$  
$V=$ Tegangan  
$I=$ Arus  
$ω=$ Kecepatan Sudut  
$R=$ Resistensi   
$L=$ Induktansi  
$J=$ Inersia  
$B=$ Viskositas  
$K_t=$ Konstanta torsi  
$K_e=$ Konstanta EMF

In [2]:
# Persamaan Listrik: V = R*I + L*di/dt + K_e*omega
omega_expr = sp.diff(theta, t)  # omega = d(theta)/dt
eq_listrik = Eq(V, R*i + L*sp.diff(i, t) + K_e*omega_expr)

# Persamaan Mekanik: T = J*d(omega)/dt + B*omega, dengan T = K_t*I
T = K_t*i  # Torsi
eq_mekanik = Eq(T, J*sp.diff(omega_expr, t) + B*omega_expr)

print("1. Persamaan Model Motor DC:")
print("Persamaan Listrik:")
sp.pprint(eq_listrik)
print("\nPersamaan Mekanik:")
sp.pprint(eq_mekanik)

1. Persamaan Model Motor DC:
Persamaan Listrik:
       d            d                
V = Kₑ⋅──(θ(t)) + L⋅──(i(t)) + R⋅i(t)
       dt           dt               

Persamaan Mekanik:
                          2       
            d            d        
Kₜ⋅i(t) = B⋅──(θ(t)) + J⋅───(θ(t))
            dt             2      
                         dt       


### 2. Transformasi Laplace pada Motor DC
- Persamaan diubah ke domain $S$ dengan asumsi kondisi awal nol.
- $I(s)$, $Ω(s)$, dan $V(s)$ adalah transformasi Laplace dari $i(t)$, $ω(t)$, dan $v(t)$

In [4]:
# Asumsi kondisi awal nol: i(0) = 0, theta(0) = 0, d(theta)/dt(0) = 0
I_s = symbols('I_s')  # Transformasi Laplace dari i(t)
Omega_s = symbols('Omega_s')  # Transformasi Laplace dari omega(t)
V_s = symbols('V_s')  # Transformasi Laplace dari v(t)

# Transformasi Laplace persamaan listrik
laplace_listrik = Eq(V_s, R*I_s + L*s*I_s + K_e*Omega_s)
print("\n2. Transformasi Laplace:")
print("Persamaan Listrik dalam domain s:")
sp.pprint(laplace_listrik)

# Transformasi Laplace persamaan mekanik
laplace_mekanik = Eq(K_t*I_s, J*s*Omega_s + B*Omega_s)
print("Persamaan Mekanik dalam domain s:")
sp.pprint(laplace_mekanik)


2. Transformasi Laplace:
Persamaan Listrik dalam domain s:
Vₛ = Iₛ⋅L⋅s + Iₛ⋅R + Kₑ⋅Ωₛ
Persamaan Mekanik dalam domain s:
Iₛ⋅Kₜ = B⋅Ωₛ + J⋅Ωₛ⋅s


### 3. Fungsi Alih Hubungan antara  kecepatan sudut ($\omega$) terhadap tegangan (V)
- Menghitung $\frac{Ω(s)}{V(s)}$ dengan menyelesaikan persamaan Laplace secara simbolik.
- Hasil disederhanakan menggunakan ***simplify***

In [5]:
# Substitusi I(s) dari persamaan mekanik ke persamaan listrik
I_s_expr = solve(laplace_mekanik, I_s)[0]  # Ekspresi I(s)
laplace_listrik_sub = laplace_listrik.subs(I_s, I_s_expr)

# Selesaikan untuk Omega(s)
sol_omega = solve(laplace_listrik_sub, Omega_s)[0]
transfer_function = sp.simplify(sol_omega / V_s)

print("\n3. Fungsi Alih Omega(s)/V(s):")
sp.pprint(transfer_function)

# Opsional: Tampilkan dalam bentuk yang lebih rapi
numerator, denominator = sp.fraction(transfer_function)
print("\nFungsi Alih dalam bentuk pecahan:")
sp.pprint(Eq(Omega_s/V_s, numerator/denominator))


3. Fungsi Alih Omega(s)/V(s):
                 Kₜ                 
────────────────────────────────────
                   2                
B⋅L⋅s + B⋅R + J⋅L⋅s  + J⋅R⋅s + Kₑ⋅Kₜ

Fungsi Alih dalam bentuk pecahan:
Ωₛ                    Kₜ                 
── = ────────────────────────────────────
Vₛ                      2                
     B⋅L⋅s + B⋅R + J⋅L⋅s  + J⋅R⋅s + Kₑ⋅Kₜ
